<a href="https://colab.research.google.com/github/costajr2019/Desafio_LLM/blob/main/desafio_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Desafio da Imersão IA

[Imersão Inteligência Artificial 2ª - Desafio da Aula 04](https://cursos.alura.com.br/imersoes/aulas/aula-4-criando-seu-proprio-chatbot-com-a-gemini-api-no-google-colab-c131)

##Objetivo

Script Python que usa o LLM da Google para analisar códigos Python (postados por alunos) localizados em um pasta chamada "respostas", separados em arquivos, comparando o código Python do aluno com um código gabarito localizado em um arquivo gabarito.txt. As respostas e orientações obtidas com a IA devem são armazenadas nos respectivos arquivos em uma pasta denominada "feedback" como comentários nos respectivos arquivos dos alunos.

##Dados

| Arquivo | Conteúdo | Local de Armazenamento |
| ---| --- | --- |
| gabarito.txt | Script que produz a solução para o problema proposto.| Pasta Raíz |
| prompt.txt | Prompt que descreve o conteúdo e a forma que o LLM deve apresentar sua análise. | Pasta Raíz
| resposta_alunoN.txt | Script do N-ésimo aluno a ser avaliado pelo LLM | Pasta "respostas"

##Saída
| Arquivo | Conteúdo | Local de Armazenamento |
|---|---| --- |
| feedback_resposta_alunoN.txt | Feedback do LLM sobre o Script do N-ésimo aluno | Pasta "feedback"

##Metodologia
1. Para cada aluno, concatenar os conteúdos dos arquivos "prompt.txt", "gabarito.txt", e "resposta_alunoN.txt" para formar uma string contendo o prompt completo.
2. Obter da LLM a avaliação do script Python de cada aluno a partir do respectivo prompt completo.
3. Gravar as avaliações em arquivos separados na pasta "feedback"


In [1]:
def concatenar_strings(conteudo_prompt, conteudo_resposta, conteudo_gabarito):
  """
  Função para concatenar o conteúdo de três strings em uma única string.

  """

  # Cria a string separadora
  separador = "-" * 50 + "\n"

  # Concatena os conteúdos dos arquivos
  texto_concatenado = (
      conteudo_prompt
      + 2 * "\n"
      + "**RESPOSTA DO ALUNO**"
      + "\n"
      + separador
      + conteudo_resposta
      + "\n"
      + "\n"
      + "**GABARITO**"
      + "\n"
      + separador
      + conteudo_gabarito
  )

  return texto_concatenado

In [2]:
import os

def processar_respostas_alunos(pasta_respostas):
  """
  Função para processar todas as respostas dos alunos em uma pasta.

  Argumentos:
    pasta_respostas: O caminho para a pasta que contém as respostas dos alunos.

  Retorno:
    Um dicionário onde as chaves são os nomes dos arquivos de resposta e os valores são as strings concatenadas contendo as respostas, prompts e gabaritos.
  """

  # Carregar o conteúdo de prompt.txt, resposta_aluno.txt e gabarito.txt
  with open("prompt.txt") as prompt_file:
    prompt = prompt_file.read()

  with open("gabarito.txt") as gabarito_file:
    gabarito = gabarito_file.read()

  respostas_processadas = {}

  for arquivo in os.listdir(pasta_respostas):
    if arquivo.endswith(".txt"):
      nome_arquivo, extensao = os.path.splitext(arquivo)

      with open(os.path.join(pasta_respostas, arquivo), 'r') as resposta_aluno_file:
        resposta_aluno = resposta_aluno_file.read()

      # Concatenar os conteúdos usando a função concatenar_arquivos
      texto_concatenado = concatenar_strings(prompt, resposta_aluno, gabarito)

      # Armazenar o texto concatenado no dicionário com o nome do arquivo como chave
      respostas_processadas[nome_arquivo] = texto_concatenado

  return respostas_processadas


In [3]:
# Exemplo de uso
pasta_respostas = "respostas"

prompts_preparados = processar_respostas_alunos(pasta_respostas)

# # Fazer algo com as respostas processadas, como imprimi-las ou salvá-las em um arquivo
# for nome_arquivo, texto_concatenado in prompts_preparados.items():
#   print(f"## Resposta para {nome_arquivo}")
#   print(texto_concatenado)

In [ ]:
for prompt in prompts_preparados:
  print(prompts_preparados[prompt])

##Escrever o feedback de cada resposta da IA no correspondente arquivo na pasta feedback.

In [5]:
!pip install -q -U google-generativeai

In [17]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key = api_key)

In [18]:
generation_config = {
    "candidate_count" : 1,
    "temperature" : 0.9, # Nível de criatividade
    #"top_p" : 0.5,
}

In [19]:
safety_settings = {
    "HARASSMENT" : "BLOCK_NONE",
    "HATE" : "BLOCK_NONE",
    "SEXUAL" : "BLOCK_NONE",
    "DANGEROUS" : "BLOCK_NONE",
}

In [20]:
model = genai.GenerativeModel(model_name= "gemini-1.0-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

In [21]:
import os

# Crie a pasta feedback se ela não existir
if not os.path.exists('feedback'):
    os.makedirs('feedback')

# Dicionário com as strings de feedback
feedback = {}

for prompt in prompts_preparados:
  response = model.generate_content(prompts_preparados[prompt])
  feedback[prompt] = response.text

# Percorra o dicionário
for key, value in feedback.items():
    # Crie um nome de arquivo usando a chave
    filename = f'feedback/{key}.txt'

    # Abra o arquivo em modo de escrita
    with open(filename, 'w') as f:
        # Escreva o feedback no arquivo
        f.write(value)